<a href="https://colab.research.google.com/github/vijishmadhavan/Lama-restore/blob/master/LaMa_restore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install requirements
%%capture

!git clone https://github.com/microsoft/Bringing-Old-Photos-Back-to-Life.git photo_restoration
%cd photo_restoration/Face_Enhancement/models/networks
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../../
%cd Global/detection_models
!git clone https://github.com/vacancy/Synchronized-BatchNorm-PyTorch
!cp -rf Synchronized-BatchNorm-PyTorch/sync_batchnorm .
%cd ../../
%cd /content/photo_restoration/Global
import os
isExist = os.path.exists("checkpoints/detection/")
if not isExist:
  os.makedirs('checkpoints/detection/')


!wget https://www.dropbox.com/s/wbwtopeevk6vnwe/FT_Epoch_latest.pt -P checkpoints/detection/
%cd /content/photo_restoration

!pip install -r requirements.txt
!git clone https://github.com/saic-mdal/lama.git
%cd /content/photo_restoration/lama


!pip install torch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0 torchtext==0.9
!pip install -r requirements.txt --quiet
!pip install wget --quiet
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html --quiet

isExist = os.path.exists("big-lama/models")
if not isExist:
  os.makedirs('big-lama/models')
%cd /content/photo_restoration/lama/big-lama/models
os.system("wget https://huggingface.co/akhaliq/lama/resolve/main/best.ckpt")
%cd /content/photo_restoration/lama/big-lama

!wget "https://www.dropbox.com/s/tihpgx1r44utu7a/config.yaml" 

!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet

In [ ]:
#@title Upload Image


%cd /content
from google.colab import files
files = files.upload()
name = list(files.keys())[0]

In [ ]:
#@title Detect scratches
%cd /content/photo_restoration/Global
import gc
import json
import os
import time
import warnings

import numpy as np
import torch
import torch.nn.functional as F
import torchvision as tv
from PIL import Image, ImageFile

from detection_models import networks
from detection_util.util import *



def data_transforms(img, full_size, method=Image.BICUBIC):
    if full_size == "full_size":
        ow, oh = img.size
        h = int(round(oh / 16) * 16)
        w = int(round(ow / 16) * 16)
        if (h == oh) and (w == ow):
            return img
        return img.resize((w, h), method)

    elif full_size == "scale_256":
        ow, oh = img.size
        pw, ph = ow, oh
        if ow < oh:
            ow = 256
            oh = ph / pw * 256
        else:
            oh = 256
            ow = pw / ph * 256

        h = int(round(oh / 16) * 16)
        w = int(round(ow / 16) * 16)
        if (h == ph) and (w == pw):
            return img
        return img.resize((w, h), method)


def scale_tensor(img_tensor, default_scale=256):
    _, _, w, h = img_tensor.shape
    if w < h:
        ow = default_scale
        oh = h / w * default_scale
    else:
        oh = default_scale
        ow = w / h * default_scale

    oh = int(round(oh / 16) * 16)
    ow = int(round(ow / 16) * 16)

    return F.interpolate(img_tensor, [ow, oh], mode="bilinear")


def blend_mask(img, mask):

    np_img = np.array(img).astype("float")

    return Image.fromarray((np_img * (1 - mask) + mask * 255.0).astype("uint8")).convert("RGB")

model = networks.UNet(
    in_channels=1,
    out_channels=1,
    depth=4,
    conv_num=2,
    wf=6,
    padding=True,
    batch_norm=True,
    up_mode="upsample",
    with_tanh=False,
    sync_bn=True,
    antialiasing=True,
)

isExist = os.path.exists("/content/photo_restoration/lama/data_for_prediction")
if not isExist:
  os.makedirs('/content/photo_restoration/lama/data_for_prediction')

output_dir = "/content/photo_restoration/lama/data_for_prediction"
input_dir = "/content/photo_restoration/lama/data_for_prediction"
## load model
#checkpoint_path = os.path.join(os.path.dirname(__file__), "checkpoints/detection/FT_Epoch_latest.pt")
checkpoint = torch.load("checkpoints/detection/FT_Epoch_latest.pt", map_location="cpu")
model.load_state_dict(checkpoint["model_state"])
print("model weights loaded")


if torch.cuda.is_available():
    model.to("cuda")
else: 
    model.cpu()
model.eval()

image_name ="/content/" + name
scratch_image = Image.open(image_name).convert("RGB")
w, h = scratch_image.size

transformed_image_PIL = data_transforms(scratch_image,full_size = "full_size")
scratch_image = transformed_image_PIL.convert("L")
scratch_image = tv.transforms.ToTensor()(scratch_image)
scratch_image = tv.transforms.Normalize([0.5], [0.5])(scratch_image)
scratch_image = torch.unsqueeze(scratch_image, 0)
_, _, ow, oh = scratch_image.shape
scratch_image_scale = scale_tensor(scratch_image)


if torch.cuda.is_available():
    scratch_image_scale = scratch_image_scale.to("cuda")
else:
    scratch_image_scale = scratch_image_scale.cpu()
with torch.no_grad():
    P = torch.sigmoid(model(scratch_image_scale))

P = P.data.cpu()
P = F.interpolate(P, [ow, oh], mode="nearest")

tv.utils.save_image(
    (P >= 0.4).float(),
    os.path.join(
        output_dir,
        image_name[9:-4] + "_mask"+ ".png",
    ),
    nrow=1,
    padding=0,
    normalize=True,
)
transformed_image_PIL.save(os.path.join(output_dir, image_name[9:-4] + ".png" ))
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#@title Download result
%cd /content/photo_restoration/lama
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil

fname = output_dir +"/"+ image_name[9:-4] + ".png" 

print('Run inpainting')
if '.jpeg' in fname:
  !PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/data_for_prediction outdir=/content/output dataset.img_suffix=.jpeg > /dev/null
elif '.jpg' in fname:
  !PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/data_for_prediction outdir=/content/output  dataset.img_suffix=.jpg > /dev/null
elif '.png' in fname:
  !PYTHONPATH=. TORCH_HOME=$(pwd) python3 bin/predict.py model.path=$(pwd)/big-lama indir=$(pwd)/data_for_prediction outdir=/content/output  dataset.img_suffix=.png > /dev/null
else:
  print(f'Error: unknown suffix .{fname.split(".")[-1]} use [.png, .jpeg, .jpg]')

from google.colab import files
files.download("/content/output/"+ image_name[9:-4] + "_mask"+ ".png") 